# Computer Vision Project: Cheese Classification

Coded by Luna McBride

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt #plotting
%matplotlib inline

from PIL import Image #Image Processing
import os #Operating System

import tensorflow as tf #Import tensorflow in order to use Keras
from keras import backend as K #Used for trying to clear memory, but I could still not clear enough RAM

from tensorflow.keras.preprocessing.sequence import pad_sequences #Add padding to help the Keras Sequencing
import tensorflow.keras.layers as L #Import the layers as L for quicker typing
from tensorflow.keras.optimizers import Adam #Pull the adam optimizer for usage
from tensorflow.keras.losses import SparseCategoricalCrossentropy #Loss function being used
from tensorflow.keras.preprocessing import image #Add image handling, because I am looking at images

from keras.models import Sequential #Sequential
from keras.layers import Conv2D, MaxPooling2D #Load 2d layers
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization #Load important layers

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cheese = pd.read_csv("../input/french-cheese-detection/fourmes_train.csv", delimiter = ";") #Put the cheese into a dataframe
cheese.head() #Take a peek at the data

In [ ]:
cheeseTest = pd.read_csv("../input/cheese-test-csv/fourmes_test.csv") #Put the cheese into a dataframe
cheeseTest.head() #Take a peek at the data

---

# Check for Null Values

In [ ]:
print(cheese.isnull().any()) #Check for null values

In [ ]:
print(cheeseTest.isnull().any()) #Check for null values

---

# Fix Test Set Column Names

In [ ]:
colNames = ["filename", "x1", "y1", "x2", "y2", "class"] #List to fix the names since I put spaces in my CSV
cols = dict(zip(cheeseTest.columns, colNames)) #Zip the current names with the new list
cheeseTest = cheeseTest.rename(columns = cols) #Rename the dataframe without the spaces
cheeseTest.head() #Take a peek at the test dataset

---

# File Name Collecting

In [ ]:
files = cheese["filename"].unique() #Get the unique file names

cheese["filenum"] = cheese["filename"].apply(lambda x: x.split(".")[0]) #Put the file number in its own column
cheese.head() #Take a peek at the dataset

---

# Image Processing

In [ ]:
#CropImage: Crops a Pil Image to specified corners (x1, y1) and (x2, y2)
#Input: the image, cornerpoints (x1, y1) and (x2, y2)
#Output: The cropped image
def cropImage(image, x1, y1, x2, y2):
    return image.crop((x1, y1, x2, y2)) #Return the image cropped via (x1, y1) and (x2, y2)

#Standardize: Standarizes images by resizing them to an x by y image
#Input: The image, variables x and y to resize the image to
#Output: The standardized image
def standardize(image, x, y):
    return image.resize((x,y), Image.ANTIALIAS) #Return the resized/standardized image

#PreprocessImages: Pulls image stats from the dataframe to collect, crop, and standardize images, saving them into a new repository
#Input: The dataframe
#Output: A list of processed images and names of their links
def preprocessImages(df, folder):
    croppedImages = [] #A list to hold processed images
    names = [] #A list to hold the link name
    
    directory = "cropped_" + folder #Give the name of the directory
    
    #If the cropped image directory does not exist, make it
    if not os.path.exists(directory):
        os.makedirs(directory) #Make the directory
        
    #For each row, collect the image, crop it, standardize it, and add it to the processed image list
    for index, row in df.iterrows():
        openImage = Image.open("../input/french-cheese-detection/{}/{}".format(folder, row["filename"])) #Open the image
        openImage = cropImage(openImage, row["x1"], row["y1"], row["x2"], row["y2"]) #Crop the image (see the cropImage method)
        stand = standardize(openImage, 60, 60) #Resize/Standardize the image (see the standardize method above)
        croppedImages.append(np.asarray(stand)) #Append the processed image to the holder list
        
        #Build the file path name
        name = "cropped_" + folder + "/" + row["filename"].split(".")[0] + str(row["x1"]) + str(row["y2"]) + "." + row["filename"].split(".")[1]
        names.append("./" + name) #Add the name to the names list
        
        #If the file does not exist, save it.
        if not os.path.isfile(name):
            d = stand.save(name) #Save the new cropped image
        
    return croppedImages, names #Return the list of processed images

In [ ]:
imagesTrain, names = preprocessImages(cheese, "fourmes_train") #Use the preprocessImages function to process images based on the database
plt.imshow(imagesTrain[3]) #Show a processed cheese image (the image processed, not the cheese. We are not working with American here)

A fine cheese indeed.

In [ ]:
imagesTest, namesTest = preprocessImages(cheeseTest, "fourmes_test") #Use the preprocessImages function to process images based on the database
plt.imshow(imagesTest[4]) #Show a processed cheese image

---

# Set up Train Test Split

In [ ]:
classTrain = cheese["class"] #Get the train class we are looking for
classTest = cheeseTest["class"] #Get the test cheese type we are looking for

classTrain = pd.get_dummies(classTrain) #Get dummies for the train set
classTest = pd.get_dummies(classTest) #Get dummies for the test set
print(classTest) #Print one of the sets

In [ ]:
cheese["name"] = names #Put the image names/urls into the cheese dataframe
train_datagen = image.ImageDataGenerator( #Build a image generator
        rescale=1./255, #Resize the images
        shear_range=0.2, #Allow shearing the images
        zoom_range=0.2, #Allow zooming
        horizontal_flip=True) #Allow horizontal flipping

train = train_datagen.flow_from_dataframe(cheese,  #Get the images from the cheese dataframe
        target_size = (60, 60), #Show that image size is to be 60x60
        batch_size = batchSize, #Set the batch size to the pre-determined size
        class_mode = "binary", #Put the mode to binary since there are two labels
        x_col = "name", #Get the image file names into x_col
        y_col = "class") #Get the label classes into y_col

In [ ]:
test_datagen = image.ImageDataGenerator(rescale=1./255) #Build a simple datagen for the test images
cheeseTest["name"] = namesTest #Add the names to the dataframe as to stream them

test = test_datagen.flow_from_dataframe(cheeseTest, #Make a flow variable for keras from the test images
                                        class_mode = "binary", #Make the mode binary since there are only two tiers
                                        x_col = "name", #Get the image names into the x_col
                                        y_col = "class", #Get the labels (cheese type) into the y_col
                                        batch_size = batchSize, #Set the batch size
                                        target_size = (60, 60)) #Show that the target image size is 60x60

Train and test are already separate. Compare to train test split syntax:

imageTrain, imageTest, classTrain, classTest = train_test_split(images, class, ...)

---

In [ ]:
tf.keras.backend.clear_session() #Clear any previous model building

epoch = 50 #Number of runs through the data
width = 60 #The width of the images
height = 60 #The height of the images
channels = 3 #The number of channels (RGB)

model = Sequential() #Add a sequential to the model
model.add(Conv2D(32, (3,3), input_shape = (width, height, channels))) #Add a convolutional image layer
model.add(BatchNormalization()) #Normalize the data
model.add(Activation("relu")) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Conv2D(32, (3, 3))) #Add a convolutional image layer
model.add(Activation("relu")) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Conv2D(64, (3, 3))) #Add a bigger convolutional image layer, layering activations
model.add(Activation("relu")) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Flatten()) #Make the layers flat to apply the characteristics into one slot
model.add(Dense(64)) #Add a dense layer to track activation
model.add(Activation("relu")) #Make the activation relu to discourage negative units
model.add(Dense(1)) #Add another dense layer to finish the lot
model.add(Activation("sigmoid")) #Make the activation sigmoid 

model.compile(loss = "binary_crossentropy", #Make the loss binary to fit with the sigmoid endpoint
              optimizer = "rmsprop", #Use root mean squared prop to optimize (adam came to similar results at much slower speeds)
              metrics = ["accuracy"]) #Track the accuracy of the model

history = model.fit_generator(train, epochs = epoch) #Fit the model to the data

In [ ]:
loss, accuracy = model.evaluate(test) #Get the loss and Accuracy based on the tests

#Print the loss and accuracy
print("Test Loss: ", loss)
print("Test Accuracy: ", accuracy)

In [ ]:
classes = ["Fourme d'Ambert", "Fourme de Montbrison"] #Adding the class names
predict = model.predict_classes(test) #Get predictions
print(predict) #Print the predictions

In [ ]:
length = len(predict) #Get the length of the predictions
true = cheeseTest["class"] #Get the true names of the cheese

#For each test cheese, print the cheese titled with the prediction
for i in range(0, length):
    plt.figure(figsize = (6, 5)) #Get the cheese picture
    plt.imshow(imagesTest[i]) #Show a processed cheese image
    plt.title("Predicted: {} \n True: {}".format(classes[predict[i][0]], true[i])) #Set the title to the predicted and true cheese names